## Keep
keep函数可以从原始数据中抽取符合指定条件的子集。
![](img/keep.png)

In [17]:
from clumper import Clumper

list_dicts = [{'a': 1}, 
              {'a': 2}, 
              {'a': 3}, 
              {'a': 4}]

(Clumper(list_dicts)
 .keep(lambda d: d['a'] >= 3)
 .collect()  #试一试去掉.collect()后的效果
)

[{'a': 3}, {'a': 4}]

可以实现缺失值处理，以不同的方式实现pandas的``.dropna()``的功能。

In [18]:
from clumper import Clumper

data = [
    {"a": 1, "b": 4},
    {"a": 2, "b": 3},
    {"a": 3, "b": 2},
    {"a": 4},
]

#只保留含有b的字段
(Clumper(data)
 .keep(lambda d: 'b' in d.keys())
 .collect()
)

[{'a': 1, 'b': 4}, {'a': 2, 'b': 3}, {'a': 3, 'b': 2}]

## Mutate
mutate可以在每条记录中，创建新字段、改写旧字段。
![](img/mutate.png)

In [23]:
from clumper import Clumper

list_dicts = [
    {'a': 1, 'b': 2},
    {'a': 2, 'b': 3, 'c':4},
    {'a': 1, 'b': 6}]

#新建了c和s字段
(Clumper(list_dicts)
 .mutate(c=lambda d: d['a'] + d['b'],
         s=lambda d: d['a'] + d['b'] + d['c'])
 .collect()
)

[{'a': 1, 'b': 2, 'c': 3, 's': 6},
 {'a': 2, 'b': 3, 'c': 5, 's': 10},
 {'a': 1, 'b': 6, 'c': 7, 's': 14}]

## Sort
sort可以实现排序
![](img/sort.png)

In [21]:
from clumper import Clumper

list_dicts = [
    {'a': 1, 'b': 2},
    {'a': 3, 'b': 3},
    {'a': 2, 'b': 1}]

(Clumper(list_dicts)
  .sort(lambda d: d['b']) #默认升序
  .collect()
)

[{'a': 2, 'b': 1}, {'a': 1, 'b': 2}, {'a': 3, 'b': 3}]

## Select
select挑选每条记录中的某个(些)字段
![](img/select.png)

In [20]:
from clumper import Clumper

list_dicts = [
    {'a': 1, 'b': 2},
    {'a': 2, 'b': 3, 'c':4},
    {'a': 1, 'b': 6}]

(Clumper(list_dicts)
 .select('a')
 .collect()
)

[{'a': 1}, {'a': 2}, {'a': 1}]

## Drop
剔除某个（些）字段。
![](img/drop.png)

In [24]:
from clumper import Clumper

list_dicts = [
    {'a': 1, 'b': 2},
    {'a': 2, 'b': 3, 'c':4},
    {'a': 1, 'b': 6}]

(Clumper(list_dicts)
 .drop('c')
 .collect()
)

[{'a': 1, 'b': 2}, {'a': 2, 'b': 3}, {'a': 1, 'b': 6}]

## GroupBy
根据某个（些）字段对数据集进行分组，得到不同Group类的集合。一般与``.agg()``方法联合使用。
![](img/groupby.png)

In [42]:
from clumper import Clumper

grade_dicts = [
    {'gender': '男', 'grade': 98, 'name': '张三'},
    {'gender': '女', 'grade': 88, 'name': '王五'},
    {'gender': '女', 'grade': 99, 'name': '赵六'},
    {'gender': '男', 'grade': 58, 'name': '李四'}]

(Clumper(grade_dicts)
 .group_by("gender")
 .groups==('gender', )
)

True

## Ungroup
GroupBy的反操作
![](img/ungroup.png)

In [44]:
from clumper import Clumper

grade_dicts = [
    {'gender': '男', 'grade': 98, 'name': '张三'},
    {'gender': '女', 'grade': 88, 'name': '王五'},
    {'gender': '女', 'grade': 99, 'name': '赵六'},
    {'gender': '男', 'grade': 58, 'name': '李四'}]

(Clumper(grade_dicts)
 .group_by("gender")
 .ungroup().groups == tuple()
)

True

## Agg
聚合描述性统计方法

agg如下图，可以理解成三个步骤，即group->split->summary
![](img/split-apply-combine.png)

常用的描述性统计函数有： ```mean、count、unqiue、n_unique、sum、min和max```

求学生的平均成绩、最优和最差成绩

In [46]:
from clumper import Clumper

grade_dicts = [
    {'gender': '男', 'grade': 98, 'name': '张三'},
    {'gender': '女', 'grade': 88, 'name': '王五'},
    {'gender': '女', 'grade': 99, 'name': '赵六'},
    {'gender': '男', 'grade': 58, 'name': '李四'}]

(Clumper(grade_dicts)
 .agg(mean_grade=('grade', 'mean'),
      max_grade=('grade', 'max'),
      min_grade=('grade', 'min'))
 .collect()
)

[{'mean_grade': 85.75, 'max_grade': 99, 'min_grade': 58}]

求男生和女生各自的平均成绩、最优和最差成绩

In [47]:
from clumper import Clumper

grade_dicts = [
    {'gender': '男', 'grade': 98, 'name': '张三'},
    {'gender': '女', 'grade': 88, 'name': '王五'},
    {'gender': '女', 'grade': 99, 'name': '赵六'},
    {'gender': '男', 'grade': 58, 'name': '李四'}]

#对
(Clumper(grade_dicts)
 .group_by('gender')
 .agg(mean_grade=('grade', 'mean'),
      max_grade=('grade', 'max'),
      min_grade=('grade', 'min'))
 .collect()
)

[{'gender': '男', 'mean_grade': 78, 'max_grade': 98, 'min_grade': 58},
 {'gender': '女', 'mean_grade': 93.5, 'max_grade': 99, 'min_grade': 88}]

## Collect
一般Clumper函数返回的结果显示为Clumper类，是看不到具体内容的。


collect作用主要是展开显示。
![](img/collect.png)

## 剔除重复
剔除重复内容
![](img/drop_duplicates.png)

In [50]:
from clumper import Clumper

data = [{"a": 1}, 
        {"a": 2}, 
        {"a": 2}]

(Clumper(data)
 .drop_duplicates()
 .collect()
 )

[{'a': 1}, {'a': 2}]